# Social Media Analysis & Misinformation Detection

**Studio Lab Quickstart** | Research Jumpstart

This notebook demonstrates social media analysis techniques including:
- Sentiment analysis
- Topic modeling
- Misinformation pattern detection
- Engagement analysis
- Network visualization

**Time to complete**: 3-4 hours

---

## 1. Setup & Imports

Import required libraries and configure settings.

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime

# NLP and text processing
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import networkx as nx

# Utilities
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ All libraries imported successfully")

## 2. Load and Explore Data

Load the sample social media dataset and examine basic statistics.

In [ ]:
# Load data
df = pd.read_csv('sample_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nDate range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"\nPlatforms: {df['platform'].value_counts().to_dict()}")
print(f"\nTotal engagement: {(df['retweets'] + df['likes'] + df['replies']).sum():,}")

In [ ]:
# Display sample posts
print("Sample posts:\n")
for idx, row in df.head(3).iterrows():
    print(f"Post {idx + 1}:")
    print(f"  Platform: {row['platform']}")
    print(f"  Text: {row['text']}")
    print(f"  Engagement: {row['likes']} likes, {row['retweets']} retweets, {row['replies']} replies")
    print()

## 3. Text Preprocessing

Clean and prepare text data for analysis.

In [ ]:
def preprocess_text(text, remove_stopwords=True):
    """
    Clean and preprocess text data.
    
    Parameters:
    - text: Input text string
    - remove_stopwords: Whether to remove stop words
    
    Returns:
    - Cleaned text string
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove user mentions and hashtags (but keep the text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Remove short words
    tokens = [word for word in tokens if len(word) > 2]
    
    return ' '.join(tokens)

# Apply preprocessing
df['text_clean'] = df['text'].apply(preprocess_text)

print("Original text:")
print(df['text'].iloc[0])
print("\nCleaned text:")
print(df['text_clean'].iloc[0])
print("\n✓ Text preprocessing complete")

## 4. Sentiment Analysis

Analyze sentiment using VADER (Valence Aware Dictionary and sEntiment Reasoner).

In [ ]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    """
    Calculate sentiment scores using VADER.
    
    Returns:
    - Dictionary with neg, neu, pos, compound scores
    """
    scores = analyzer.polarity_scores(text)
    return scores

def classify_sentiment(compound_score):
    """
    Classify sentiment based on compound score.
    
    Returns:
    - 'positive', 'negative', or 'neutral'
    """
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Calculate sentiment scores
sentiment_scores = df['text'].apply(get_sentiment)
df['sentiment_compound'] = sentiment_scores.apply(lambda x: x['compound'])
df['sentiment_pos'] = sentiment_scores.apply(lambda x: x['pos'])
df['sentiment_neg'] = sentiment_scores.apply(lambda x: x['neg'])
df['sentiment_neu'] = sentiment_scores.apply(lambda x: x['neu'])
df['sentiment_label'] = df['sentiment_compound'].apply(classify_sentiment)

print("Sentiment Distribution:")
print(df['sentiment_label'].value_counts())
print(f"\nAverage compound score: {df['sentiment_compound'].mean():.3f}")

In [ ]:
# Visualize sentiment distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sentiment counts
sentiment_counts = df['sentiment_label'].value_counts()
axes[0].bar(sentiment_counts.index, sentiment_counts.values, color=['green', 'gray', 'red'])
axes[0].set_title('Sentiment Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Sentiment')
axes[0].set_ylabel('Count')
axes[0].grid(axis='y', alpha=0.3)

# Compound score distribution
axes[1].hist(df['sentiment_compound'], bins=20, color='skyblue', edgecolor='black')
axes[1].axvline(x=0, color='red', linestyle='--', label='Neutral threshold')
axes[1].set_title('Compound Sentiment Score Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Compound Score')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Sentiment analysis complete")

## 5. Topic Modeling

Discover main topics using Latent Dirichlet Allocation (LDA).

In [ ]:
# Prepare data for topic modeling
n_topics = 3
n_top_words = 10

# Create document-term matrix
vectorizer = CountVectorizer(max_features=100, min_df=1, max_df=0.8)
doc_term_matrix = vectorizer.fit_transform(df['text_clean'])

# Train LDA model
lda_model = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=10,
    learning_method='online',
    random_state=42
)

lda_output = lda_model.fit_transform(doc_term_matrix)

# Assign dominant topic to each document
df['dominant_topic'] = lda_output.argmax(axis=1)

print(f"Trained LDA model with {n_topics} topics\n")
print("Topic distribution:")
print(df['dominant_topic'].value_counts().sort_index())

In [ ]:
# Display top words for each topic
feature_names = vectorizer.get_feature_names_out()

print("Top words per topic:\n")
for topic_idx, topic in enumerate(lda_model.components_):
    top_words_idx = topic.argsort()[-n_top_words:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")
    print()

print("✓ Topic modeling complete")

## 6. Engagement Analysis

Analyze engagement patterns and identify viral content.

In [ ]:
# Calculate total engagement
df['total_engagement'] = df['retweets'] + df['likes'] + df['replies']

# Identify viral content (top 20%)
engagement_threshold = df['total_engagement'].quantile(0.80)
df['is_viral'] = df['total_engagement'] >= engagement_threshold

print(f"Engagement threshold for viral content: {engagement_threshold}")
print(f"\nViral posts: {df['is_viral'].sum()} ({df['is_viral'].mean()*100:.1f}%)")
print(f"\nTop 3 most engaging posts:")
print(df.nlargest(3, 'total_engagement')[['text', 'total_engagement', 'sentiment_label']])

In [ ]:
# Visualize engagement by platform
fig, ax = plt.subplots(figsize=(10, 6))

platform_engagement = df.groupby('platform')['total_engagement'].sum().sort_values(ascending=False)
platform_engagement.plot(kind='bar', ax=ax, color='coral')
ax.set_title('Total Engagement by Platform', fontsize=14, fontweight='bold')
ax.set_xlabel('Platform')
ax.set_ylabel('Total Engagement')
ax.grid(axis='y', alpha=0.3)

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Engagement vs sentiment correlation
correlation = df['total_engagement'].corr(df['sentiment_compound'])
print(f"\nCorrelation between engagement and sentiment: {correlation:.3f}")
print("✓ Engagement analysis complete")

## 7. Misinformation Pattern Detection

Detect potential misinformation indicators using pattern matching.

In [ ]:
def detect_misinformation_patterns(text):
    """
    Detect misinformation indicators in text.
    
    Returns:
    - Dictionary with indicator flags and risk score
    """
    indicators = {}
    
    # 1. ALL CAPS words
    caps_words = re.findall(r'\b[A-Z]{3,}\b', text)
    indicators['excessive_caps'] = len(caps_words) > 2
    indicators['caps_count'] = len(caps_words)
    
    # 2. Excessive punctuation
    exclamation_count = text.count('!')
    question_count = text.count('?')
    indicators['excessive_punctuation'] = (exclamation_count > 2) or (question_count > 2)
    
    # 3. Urgency language
    urgency_keywords = ['breaking', 'urgent', 'immediately', 'right now', 'before its too late']
    indicators['urgency_language'] = any(keyword in text.lower() for keyword in urgency_keywords)
    
    # 4. Vague sources
    vague_sources = ['they', 'mainstream media', 'they dont want', 'theyre hiding']
    indicators['vague_sources'] = any(source in text.lower() for source in vague_sources)
    
    # 5. Call to action
    cta_keywords = ['share', 'retweet', 'wake up', 'open your eyes', 'do your own research']
    indicators['call_to_action'] = any(keyword in text.lower() for keyword in cta_keywords)
    
    # 6. Conspiracy keywords
    conspiracy_keywords = ['truth', 'coverup', 'cover-up', 'conspiracy', 'deep state', 'globalist']
    indicators['conspiracy_language'] = any(keyword in text.lower() for keyword in conspiracy_keywords)
    
    # Calculate risk score (0-10)
    risk_score = sum([
        indicators['excessive_caps'] * 2,
        indicators['excessive_punctuation'] * 1,
        indicators['urgency_language'] * 2,
        indicators['vague_sources'] * 2,
        indicators['call_to_action'] * 2,
        indicators['conspiracy_language'] * 1
    ])
    
    indicators['risk_score'] = min(risk_score, 10)  # Cap at 10
    
    return indicators

# Apply misinformation detection
misinfo_results = df['text'].apply(detect_misinformation_patterns)
df['risk_score'] = misinfo_results.apply(lambda x: x['risk_score'])
df['has_caps'] = misinfo_results.apply(lambda x: x['excessive_caps'])
df['has_urgency'] = misinfo_results.apply(lambda x: x['urgency_language'])
df['has_vague_sources'] = misinfo_results.apply(lambda x: x['vague_sources'])

# Classify risk level
def classify_risk(score):
    if score <= 2:
        return 'low'
    elif score <= 5:
        return 'medium'
    elif score <= 8:
        return 'high'
    else:
        return 'critical'

df['risk_level'] = df['risk_score'].apply(classify_risk)

print("Risk Level Distribution:")
print(df['risk_level'].value_counts())
print(f"\nAverage risk score: {df['risk_score'].mean():.2f}")
print(f"\nHigh-risk posts (score > 5): {(df['risk_score'] > 5).sum()}")

In [ ]:
# Visualize risk distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Risk level distribution
risk_order = ['low', 'medium', 'high', 'critical']
risk_counts = df['risk_level'].value_counts().reindex(risk_order, fill_value=0)
colors = ['green', 'yellow', 'orange', 'red']
axes[0].bar(risk_counts.index, risk_counts.values, color=colors)
axes[0].set_title('Misinformation Risk Level Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Risk Level')
axes[0].set_ylabel('Count')
axes[0].grid(axis='y', alpha=0.3)

# Risk score histogram
axes[1].hist(df['risk_score'], bins=11, range=(0, 10), color='salmon', edgecolor='black')
axes[1].axvline(x=5, color='red', linestyle='--', label='High risk threshold')
axes[1].set_title('Risk Score Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Risk Score')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Show high-risk posts
high_risk = df[df['risk_score'] > 5].sort_values('risk_score', ascending=False)
if len(high_risk) > 0:
    print("\nHigh-risk posts:")
    for idx, row in high_risk.iterrows():
        print(f"\nRisk Score: {row['risk_score']}")
        print(f"Text: {row['text']}")

print("\n✓ Misinformation pattern detection complete")

## 8. Summary Statistics

Generate comprehensive summary of findings.

In [ ]:
print("=" * 60)
print("SOCIAL MEDIA ANALYSIS SUMMARY")
print("=" * 60)

print(f"\n📊 DATASET OVERVIEW")
print(f"  Total posts analyzed: {len(df)}")
print(f"  Date range: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}")
print(f"  Platforms: {', '.join(df['platform'].unique())}")

print(f"\n😊 SENTIMENT ANALYSIS")
sentiment_pct = df['sentiment_label'].value_counts(normalize=True) * 100
for sentiment in ['positive', 'neutral', 'negative']:
    pct = sentiment_pct.get(sentiment, 0)
    print(f"  {sentiment.capitalize()}: {pct:.1f}%")
print(f"  Average sentiment: {df['sentiment_compound'].mean():.3f}")

print(f"\n📝 TOPIC MODELING")
print(f"  Number of topics identified: {n_topics}")
topic_dist = df['dominant_topic'].value_counts().sort_index()
for topic_id, count in topic_dist.items():
    print(f"  Topic {topic_id}: {count} posts ({count/len(df)*100:.1f}%)")

print(f"\n🚀 ENGAGEMENT METRICS")
print(f"  Total engagement: {df['total_engagement'].sum():,}")
print(f"  Average engagement per post: {df['total_engagement'].mean():.1f}")
print(f"  Viral posts (top 20%): {df['is_viral'].sum()}")
print(f"  Most engaged platform: {df.groupby('platform')['total_engagement'].sum().idxmax()}")

print(f"\n⚠️ MISINFORMATION RISK")
risk_dist = df['risk_level'].value_counts()
for level in ['low', 'medium', 'high', 'critical']:
    count = risk_dist.get(level, 0)
    pct = count / len(df) * 100
    print(f"  {level.capitalize()} risk: {count} posts ({pct:.1f}%)")
print(f"  Average risk score: {df['risk_score'].mean():.2f} / 10")

print(f"\n🔍 KEY FINDINGS")
print(f"  • Most common sentiment: {df['sentiment_label'].mode()[0]}")
print(f"  • Highest engagement post had {df['total_engagement'].max():,} interactions")
print(f"  • {(df['risk_score'] > 5).sum()} posts flagged as high-risk")
print(f"  • Sentiment-engagement correlation: {df['total_engagement'].corr(df['sentiment_compound']):.3f}")

print("\n" + "=" * 60)
print("Analysis complete! Review visualizations above for details.")
print("=" * 60)

## 9. Next Steps

### Experiment with This Notebook

1. **Modify parameters**:
   - Adjust number of topics in LDA
   - Change risk score thresholds
   - Modify sentiment classification ranges

2. **Add your own data**:
   - Replace `sample_data.csv` with your dataset
   - Ensure same column structure
   - Increase dataset size

3. **Extend the analysis**:
   - Add temporal analysis (trends over time)
   - Create word clouds for different topics
   - Analyze user networks
   - Compare platforms

### Scale to Production

When ready for real social media data:

1. Review the [Unified Studio README](../unified-studio/README.md)
2. Set up AWS account
3. Access real Twitter/Reddit datasets
4. Use Amazon Comprehend for advanced NLP
5. Deploy with Amazon Bedrock for AI content analysis

### Learn More

- [VADER Sentiment Analysis Paper](https://ojs.aaai.org/index.php/ICWSM/article/view/14550)
- [LDA Topic Modeling Tutorial](https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html)
- [First Draft Misinformation Research](https://firstdraftnews.org/)
- [Computational Social Science](https://compsocialscience.github.io/)

---

**Questions or issues?** Open an issue at https://github.com/research-jumpstart/research-jumpstart/issues

*Last updated: 2025-11-09*